### **Fact orders**

Data reading

In [0]:
df = spark.sql("select * from databricks_catalog.silver.orders_silver")
df.display()

In [0]:
df_dimcustomer = spark.sql("select DimCustomerKey, customer_id as dim_customer_id from databricks_catalog.gold.dimcustomers")

df_dimproduct = spark.sql("select product_id as DimProductKey, product_id as dim_product_id from databricks_catalog.gold.dimproducts")

In [0]:
df_dimproduct.display()

In [0]:
%sql
select * from databricks_catalog.gold.dimproducts

Fact Table

In [0]:
df_fact = df.join(df_dimcustomer, df['customer_id'] == df_dimcustomer['dim_customer_id'], how='left').join(df_dimproduct, df['product_id'] == df_dimproduct['dim_product_id'], how='left')

df_fact_new = df_fact.drop('dim_customer_id', 'dim_product_id', 'customer_id', 'product_id')

In [0]:
df_fact_new.display()

### **Upsert on fact table**

In [0]:
from delta.tables import *

In [0]:
if spark.catalog.tableExists("databricks_catalog.gold.factorders"):
  
  dlt_obj = DeltaTable.forName(spark, 'databricks_catalog.gold.factorders')

  dlt_obj.alias("trg").merge(df_fact_new.alias("src"), "trg.order_id = src.order_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
  df_fact_new.write.format("delta").option('path','abfss://gold@databricksstorage222.dfs.core.windows.net/factorders').saveAsTable("databricks_catalog.gold.factorders")

In [0]:
%sql
select * from databricks_catalog.gold.factorders